<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# 对象跟随- 进阶优化版

在这个历程中，我们将展示如何使用JetBot跟踪对象!
我们将使用一个在[COCO dataset](http://cocodataset.org)上训练过的预训练神经网络来检测90个不同的常见对象。这些包括

* Person(人) (index 0)
* Cup(杯子) (index 47)

不管怎样，我们开始吧。首先，我们想导入``ObjectDetector``类，它使用我们预先训练的SSD引擎。还有许多其他类(你可以检查[这个文件](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbt)以获得类索引的完整列表)。该模型来自[TensorFlow对象检测API](https://github.com/tensorflow/models/tree/master/research/object_detection)，该API还为定制任务的对象检测器培训提供实用程序!一旦模型被训练，我们使用NVIDIA TensorRT对Jetson Nano进行优化。这使得网络非常快，能够实时控制Jetbot!但是，我们不会在这个笔记本中运行所有的培训和优化步骤。不管怎样，我们开始吧。首先，我们要导入“ObjectDetector”类，该类使用我们预先训练的SSD引擎。

### 单张相机图像检测

In [ ]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

在内部，``ObjectDetector ``类使用TensorRT Python API来执行我们提供的引擎。它还负责对神经网络的输入进行预处理，以及对检测到的对象进行解析。目前，它只适用于使用``jetbotssd_tensorrt``包创建的引擎。该包具有将模型从TensorFlow对象检测API转换为优化的TensorRT引擎的实用程序。
接下来，让我们初始化相机。我们的检测器需要300x300像素的输入，所以我们会在创建相机时设置这个。

>在内部，Camera类使用GStreamer来利用Jetson Nano的图像信号处理器(ISP),获取图像的速度是非常可观的.

>并且还从CPU卸载了大量的大小计算。

In [2]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

现在，让我们使用一些摄像机输入来执行我们的网络。在默认情况下，``ObjectDetector``类期望相机生成的格式为``bgr8``。
然而，如果输入格式不同，可以覆盖默认的预处理函数。

如果相机的视场中有任何COCO对象，它们现在应该存储在``detections``变量中。
### 在文本区域显示检测的对象

我们将使用下面的代码打印出检测到的对象。

In [8]:
detections = model(camera.value)

print(detections)

[[{'label': 1, 'confidence': 0.9264369606971741, 'bbox': [0.006075531244277954, 1.9103288650512695e-05, 0.9953758716583252, 0.9970616102218628]}]]


In [9]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

Textarea(value="[[{'label': 1, 'confidence': 0.9264369606971741, 'bbox': [0.006075531244277954, 1.910328865051…

您应该看到每个图像中检测到的每个对象的标签、置信度和边框。在这个例子中只有一个图像(我们的相机)。

要打印第一张图像中检测到的第一个对象，我们可以调用以下命令

>如果没有检测到对象，这可能会抛出一个错误

In [10]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

{'label': 1, 'confidence': 0.9264369606971741, 'bbox': [0.006075531244277954, 1.9103288650512695e-05, 0.9953758716583252, 0.9970616102218628]}


> 如果没有检测到对象，这里可能会抛出一个错误

# 控制机器人跟随中心物体

现在，我们希望机器人跟随指定类的对象。为此，我们将做以下工作

1.  检测与指定类匹配的对象
2.  选择距离相机视野中心最近的物体，这是目标物体
3.  引导机器人向目标物体移动，否则会产生漂移
4.  如果我们被障碍物挡住了，向左转

我们还将创建一些小部件，用于控制目标对象标签、机器人速度和转弯增益，根据目标对象和机器人视野中心之间的距离来控制机器人转弯的速度。

首先，让我们加载碰撞检测模型。为了方便起见，预训练的模型存储在程序这个目录中.

In [11]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

# collision_model = torchvision.models.alexnet(pretrained=False)
# collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
# collision_model.load_state_dict(torch.load('best_model.pth'))
# device = torch.device('cuda')
# collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    #图片缩放至224,224对比224,244的避障模型
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

创建驱动电机的robot实例.

In [12]:
from jetbot import Robot

robot = Robot()

### 导入PID驱动模块，创建PID控制器实例，初始化相应控制变量

In [13]:
import PID

global follow_speed
follow_speed = 0.5
global turn_gain
turn_gain = 1.7
global follow_speed_pid, follow_speed_pid_model
follow_speed_pid_model = 1
# follow_speed_pid = PID.PositionalPID(3, 0, 0)
follow_speed_pid = PID.PositionalPID(1.5, 0, 0.05)
global turn_gain_pid
turn_gain_pid = PID.PositionalPID(0.15, 0, 0.05)
global object_yservo_pid
object_yservo_pid = PID.PositionalPID(3, 0.5, 0)

当我们需要跟随人时我们可以通过下面单元格先把云台的角度自动调节到Jetbot跟随人时的仰角

In [15]:
from servoserial import ServoSerial
servo_device = ServoSerial() 
# if label_widget.value == 1:
#servo_device.Servo_serial_double_control(1, 2048, 2, 2500)

serial Open!


最后，让我们显示所有控件小部件，并将网络执行功能连接到相机更新。

In [17]:
from jetbot import bgr8_to_jpeg

image_widget = widgets.Image(format='jpeg', width=300, height=300)
#将下面这条语句的value值改为检测目标物体的对象值，取值范围0-99，例如跟随对象是Person(人) (index 1)，需要将value的值改为1
label_widget = widgets.IntText(value=1, description='tracked label')

display(widgets.VBox([
    widgets.HBox([image_widget]),
    label_widget,
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """计算对象的中心x、y坐标"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """计算二维向量的长度"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """查找最接近图像中心的检测"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    global follow_speed
    global turn_gain
    global follow_speed_pid
    target_value_speed = 0
    #更新图片值
    image = change['new']

    #执行冲突模型以确定是否阻塞
    # execute collision model to determine if blocked
    # collision_output = collision_model(preprocess(image)).detach().cpu()
    #应用“softmax”函数对输出向量进行标准化，使其和为1(这使其成为一个概率分布)
    # prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    # blocked_widget.value = prob_blocked
    
    #堵塞不执行后面的对象检测程序,直接return进入下一轮数据刷新
    # turn left if blocked
    # if prob_blocked > 0.5:
    #     robot.left(0.3)
    #     image_widget.value = bgr8_to_jpeg(image)
    #     return
        
    # compute all detected objects
    detections = model(image)
    #首先把所出现的非跟踪的对象用蓝色的线条标注出来
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (255, 0, 0), 2)
    #选择你所要跟踪的对象,即选择label_widget的值,1是person,人
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    #然后再把要跟踪的对象用绿色的线条标注出
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (0, 255, 0), 4)

    # otherwise go forward if no target detected
    '''
    blob:https---------html5中blob对象在赋给video标签后生成的一串标记
    '''
    if det is None:
        # robot.forward(float(follow_speed))
        robot.stop()
    # otherwsie steer towards target
    else:
        #控制将机器人向前移动，并控制成比例的目标与中心的x距离
        # move robot forward and steer proportional target's x-distance from center
        center = detection_center(det)

        #跟随速度PID调节
        follow_speed_pid.SystemOutput = 90000 * (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
        if label_widget.value == 1:
            follow_speed_pid.SetStepSignal(30000)
        elif label_widget.value == 53 or label_widget.value == 55: # 53:Apple 55:orange
            follow_speed_pid.SetStepSignal(10000)
        else:
            follow_speed_pid.SetStepSignal(10000)
        follow_speed_pid.SetInertiaTime(0.2, 0.1)

        #转向增益PID调节
        turn_gain_pid.SystemOutput = center[0]
        turn_gain_pid.SetStepSignal(0)
        turn_gain_pid.SetInertiaTime(0.2, 0.1)

        #将转向增益限制在有效范围内
        if label_widget.value == 1:
            target_value_turn_gain = 0.6 + abs(turn_gain_pid.SystemOutput)
        elif label_widget.value == 53 or label_widget.value == 55:
            target_value_turn_gain = 0.5 + abs(turn_gain_pid.SystemOutput)
        else:
            target_value_turn_gain = 0.5 + abs(turn_gain_pid.SystemOutput)
        if target_value_turn_gain < 0:
            target_value_turn_gain = 0
        elif target_value_turn_gain > 2:
            target_value_turn_gain = 2

        #将输出电机速度保持在有效行驶范围内
        target_value_speed = 0.46 + follow_speed_pid.SystemOutput / 90000
        target_value_speedl = target_value_speed + target_value_turn_gain * center[0]
        target_value_speedr = target_value_speed - target_value_turn_gain * center[0]
        if target_value_speedl<0.5:
            target_value_speedl=0
        elif target_value_speedl>1:
            target_value_speedl = 1
        if target_value_speedr<0.5:
            target_value_speedr=0
        elif target_value_speedr>1:
            target_value_speedr = 1

        robot.set_motors(target_value_speedl, target_value_speedr)
        
        if label_widget.value != 1:
            #垂直角度相机PID调节
            object_yservo_pid.SystemOutput =bbox[1]*300+150*(bbox[3] - bbox[1])
            # print('y轴对象中心位置值:')
            # print(object_yservo_pid.SystemOutput)
            object_yservo_pid.SetStepSignal(150)
            object_yservo_pid.SetInertiaTime(0.2, 0.1)
            #将垂直角度相机角度限制在指定范围内
            target_value_object_yservo = int(2048 + object_yservo_pid.SystemOutput)
            servo_device.Servo_serial_control(2, target_value_object_yservo)
    # 更新图像显示至小部件
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

调用下面的块将执行函数连接到每个摄像机帧更新。

In [18]:
#camera.unobserve_all()
#camera.observe(execute, names='value')

import threading
import inspect
import ctypes
'''以下为定义关闭线程函数'''
def _async_raise(tid, exctype):
  """raises the exception, performs cleanup if needed"""
  tid = ctypes.c_long(tid)
  if not inspect.isclass(exctype):
    exctype = type(exctype)
  res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
  if res == 0:
    raise ValueError("invalid thread id")
  elif res != 1:
    # """if it returns a number greater than one, you're in trouble,
    # and you should call it again with exc=NULL to revert the effect"""
    ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
    raise SystemError("PyThreadState_SetAsyncExc failed")
def stop_thread(thread):
  _async_raise(thread.ident, SystemExit)
'''线程1的函数，里面不断调用检测函数'''
def test():
    while True:
        execute({'new': camera.value})        

        
thread1 = threading.Thread(target=test)
thread1.setDaemon(False)
thread1.start()

如果机器人没有被阻挡，你应该可以看到蓝色的方框围绕着被探测到的物体,目标对象(机器人跟随的对象)将显示为绿色。
当目标被发现时，机器人应该转向目标。
您可以调用下面的代码块来手动断开与摄像机的处理并停止机器人。

In [ ]:
import time
stop_thread(thread1)
camera.unobserve_all()
time.sleep(1.0)
robot.stop()